<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

    100% |████████████████████████████████| 87.4MB 327kB/s 
    100% |████████████████████████████████| 23.8MB 1.6MB/s 
    100% |████████████████████████████████| 2.7MB 10.8MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import turicreate as tc

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Price_frequency/master/20190207_transactions.json'

In [0]:
transactions = pd.read_json(url, lines= True)

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"
